# 지하철 만족도에 영향을 끼칠 수 있는 변수 조사

In [1]:
import pandas as pd

# 지하철 사고 통계

철도사고, 운행장애 등 연단위 상세 정보 데이터

서울시 내에서 일어난 사고만 취합한 뒤

승객들에게 영향을 주지 않는 사고사례 (지연열차수, 지연시간이 없는 사고) 제외

In [2]:
ac_2020 = pd.read_csv('./한국교통안전공단_국내 철도사고, 운행장애 발생 정보_20201231/2020년 철도사고, 운행장애 정보.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
ac_2019 = pd.read_csv('./한국교통안전공단_국내 철도사고, 운행장애 발생 정보_20201231/2019년 철도사고, 운행장애 정보.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
ac_2018 = pd.read_csv('./한국교통안전공단_국내 철도사고, 운행장애 발생 정보_20201231/2018년 철도사고, 운행장애 정보.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')

# 1. 자치구 간 차이로 상관계수를 측정

# 2020년

In [3]:
ac_2020.head()

,일자,요일,철도구분,사고 종류,철도사고 종류,주원인,근본원인별 그룹,근본원인별 원인,근본원인별 상세,위험원,...,지진,바람,장소유형A,장소유형B,선로유형,신호시스템 유형,건널목사고 장소,건널목사고 차종,위험물관련 원인,위험물관련 종류
0,2020-12-31,목,일반철도,장애(지연),NaN,NaN,기술적요인,열차차량설비,단품불량/부품노후,NaN,...,NaN,NaN,역간,기타,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,목,일반철도,장애(지연),NaN,NaN,기술적요인,열차차량설비,단품불량/부품노후,NaN,...,NaN,NaN,역(역구내선로),기타,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-30,수,고속철도,장애(지연),NaN,NaN,외적요인,환경요인,강설,NaN,...,NaN,NaN,역(역구내선로),기타,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-30,수,고속철도,장애(지연),NaN,NaN,기술적요인,열차차량설비,설계/제작,NaN,...,NaN,NaN,NaN,NaN,NaN,차상신호,NaN,NaN,NaN,NaN
4,2020-12-30,수,일반철도,사고,교통사상사고직원,직원(교통)미승인작업,인적요인,작업자,규정위반,치임(교통)직원,...,NaN,NaN,역간,기타,NaN,NaN,NaN,NaN,NaN,NaN


지하철 이용 만족도와 관련이 있을 가능성이 있는 데이터만 취합

In [4]:
acc2020_df = ac_2020.loc[:,['사고 종류','행정구역','합계지연열차수','철도구분','합계최대지연시간(분)']]
acc2020_df.head()

,사고 종류,행정구역,합계지연열차수,철도구분,합계최대지연시간(분)
0,장애(지연),경상북도 예천군 역전길 4 (예천읍 2127) 예천역,1.0,일반철도,11
1,장애(지연),세종특별자치시 청연로 90 (부강면 401211) 부강역,1.0,일반철도,41
2,장애(지연),충청북도 청주시 흥덕구 오송가락로 123 (오송읍 37031) 오송역,5.0,고속철도,24
3,장애(지연),대전광역시 동구 중앙로 215 (정동 11) 대전역,1.0,고속철도,20
4,사고,충청남도 천안시 동남구 대흥로 239 (대흥동 571) 천안역,24.0,일반철도,340


행정구역 전처리 - 서울특별시만 남기기

In [5]:
# 행정구역을 '시도'와 '군구'로 나누기
acc2020_df['시도'] = acc2020_df['행정구역'].apply(lambda v:list(str(v).split())[0])
acc2020_df['군구'] = acc2020_df['행정구역'].apply(lambda v:list(str(v).split())[1])
acc2020_df.head()

,사고 종류,행정구역,합계지연열차수,철도구분,합계최대지연시간(분),시도,군구
0,장애(지연),경상북도 예천군 역전길 4 (예천읍 2127) 예천역,1.0,일반철도,11,경상북도,예천군
1,장애(지연),세종특별자치시 청연로 90 (부강면 401211) 부강역,1.0,일반철도,41,세종특별자치시,청연로
2,장애(지연),충청북도 청주시 흥덕구 오송가락로 123 (오송읍 37031) 오송역,5.0,고속철도,24,충청북도,청주시
3,장애(지연),대전광역시 동구 중앙로 215 (정동 11) 대전역,1.0,고속철도,20,대전광역시,동구
4,사고,충청남도 천안시 동남구 대흥로 239 (대흥동 571) 천안역,24.0,일반철도,340,충청남도,천안시


In [6]:
# '시도'가 서울특별시인 것만 남기기
acc2020_seoul = acc2020_df[(acc2020_df['시도']=='서울특별시')]
acc2020_seoul.head()

,사고 종류,행정구역,합계지연열차수,철도구분,합계최대지연시간(분),시도,군구
16,장애(지연),서울특별시 용산구 한강대로 23길 55,1.0,일반철도,58,서울특별시,용산구
18,사고,서울특별시 송파구 올림픽로 지하265(신천동) 2호선 잠실역,NaN,도시철도,0,서울특별시,송파구
26,장애(지연),서울특별시 중구 세종대로 자하2,64.0,도시철도,95,서울특별시,중구
30,장애(지연),서울특별시 용산구 청파로 378 (동자동 43205) 한국철도공사,3.0,고속철도,23,서울특별시,용산구
36,사고,서울특별시 강남구 선릉로 228 (대치동 101128) 한티역,NaN,도시철도,0,서울특별시,강남구


In [7]:
# 고속철도는 제외하기
acc2020_seoul = acc2020_df[(acc2020_df['철도구분']=='일반철도')|(acc2020_df['철도구분']=='도시철도')]
acc2020_seoul.head()

,사고 종류,행정구역,합계지연열차수,철도구분,합계최대지연시간(분),시도,군구
0,장애(지연),경상북도 예천군 역전길 4 (예천읍 2127) 예천역,1.0,일반철도,11,경상북도,예천군
1,장애(지연),세종특별자치시 청연로 90 (부강면 401211) 부강역,1.0,일반철도,41,세종특별자치시,청연로
4,사고,충청남도 천안시 동남구 대흥로 239 (대흥동 571) 천안역,24.0,일반철도,340,충청남도,천안시
5,장애(지연),경상남도 밀양시 삼랑진로 119 (삼랑진읍 1015),2.0,일반철도,117,경상남도,밀양시
8,사고,경상북도 경주시 원화로 266 (성동동 40) 경주역,NaN,일반철도,0,경상북도,경주시


합계 지연열차수, 합계 최대지연시간을 자치구 별로 합산

In [8]:
acc2020_time = acc2020_seoul.loc[:,['합계지연열차수','합계최대지연시간(분)','군구']]
acc2020_time.groupby(['군구'],as_index=False).sum()

,군구,합계지연열차수,합계최대지연시간(분)
0,가평군,7.0,96
1,강남구,29.0,37
2,강릉시,23.0,130
3,강북구,31.0,15
4,강서구,0.0,0
...,...,...,...
81,충주시,1.0,55
82,태백시,1.0,28
83,평택시,1.0,27
84,포항시,0.0,0


# 2020 지하철 이용 만족도 통계

상관계수 비교를 위해 해당 연도(2020) 만족도 데이터를 가져옴

In [9]:
AQ8A2 = pd.read_csv('./2020 자치구별 평균 만족도.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
AQ8A2.columns=['군구','만족도']
AQ8A2.head()

,군구,만족도
0,강남구,3.976471
1,강동구,3.944121
2,강북구,3.901693
3,강서구,3.988815
4,관악구,3.921301


두 데이터 프레임을 '군구' column 기준으로 병합

결측치는 0 처리

In [10]:
acc2020_merge = pd.merge(AQ8A2,acc2020_time.groupby(['군구'],as_index=False).sum(),how='left',on='군구')
acc2020_merge = acc2020_merge.fillna(0)
acc2020_merge

,군구,만족도,합계지연열차수,합계최대지연시간(분)
0,강남구,3.976471,29.0,37.0
1,강동구,3.944121,0.0,0.0
2,강북구,3.901693,31.0,15.0
3,강서구,3.988815,0.0,0.0
4,관악구,3.921301,0.0,0.0
5,광진구,3.965879,0.0,0.0
6,구로구,3.961316,15.0,79.0
7,금천구,3.846832,8.0,37.0
8,노원구,3.923875,2.0,43.0
9,도봉구,3.881669,1.0,7.0


In [11]:
acc2020_merge.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.193503,0.129108
합계지연열차수,-0.193503,1.000000,0.499465
합계최대지연시간(분),0.129108,0.499465,1.000000


2020년도 만족도(AQ8A2) 와의 상관관계

합계지연열차수  -0.193503

합계최대지연시간(분)  0.129108	

# 2019년

In [12]:
#지하철 이용 만족도와 관련이 있을 가능성이 있는 데이터만 취합

acc2019_df = ac_2019.loc[:,['사고 종류','행정구역','합계지연열차수','철도구분','합계최대지연시간(분)']]
acc2019_df.head()

# 행정구역을 '시도'와 '군구'로 나누기

acc2019_df['시도'] = acc2019_df['행정구역'].apply(lambda v:list(str(v).split())[0])
acc2019_df['군구'] = acc2019_df['행정구역'].apply(lambda v:list(str(v).split())[1])

# '시도'가 서울특별시인 것만 남기기

acc2019_seoul = acc2019_df[(acc2019_df['시도']=='서울특별시')]

# 고속철도는 제외하기

acc2019_seoul = acc2019_df[(acc2019_df['철도구분']=='일반철도')|(acc2019_df['철도구분']=='도시철도')]

# 합계 지연열차수, 합계 최대지연시간을 자치구 별로 합산

acc2019_time = acc2019_seoul.loc[:,['합계지연열차수','합계최대지연시간(분)','군구']]
acc2019_time.groupby(['군구'],as_index=False).sum()

# 상관계수 비교를 위해 해당 연도(2019) 만족도 데이터를 가져옴

aq28_1a2 = pd.read_csv('./2019 자치구별 평균 만족도.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
aq28_1a2.columns=['군구','만족도']

# 두 데이터 프레임을 '군구' column 기준으로 병합

# 결측치는 0 처리

acc2019_merge = pd.merge(aq28_1a2,acc2019_time.groupby(['군구'],as_index=False).sum(),how='left',on='군구')
acc2019_merge = acc2019_merge.fillna(0)
acc2019_merge

acc2019_merge.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.202766,0.003121
합계지연열차수,0.202766,1.000000,0.870918
합계최대지연시간(분),0.003121,0.870918,1.000000


2019년도 만족도(aq28_1a2) 와의 상관관계

합계지연열차수  0.202766		

합계최대지연시간(분)  0.003121

# 2018년

In [13]:
#지하철 이용 만족도와 관련이 있을 가능성이 있는 데이터만 취합

acc2018_df = ac_2018.loc[:,['사고 종류','행정구역','합계지연열차수','철도구분','합계최대지연시간(분)']]
acc2018_df.head()

# 행정구역을 '시도'와 '군구'로 나누기

acc2018_df['시도'] = acc2018_df['행정구역'].apply(lambda v:list(str(v).split())[0])
acc2018_df['군구'] = acc2018_df['행정구역'].apply(lambda v:list(str(v).split())[1])
acc2018_df.head()

# '시도'가 서울특별시인 것만 남기기

acc2018_seoul = acc2018_df[(acc2018_df['시도']=='서울특별시')]
acc2018_seoul.head()

# 고속철도는 제외하기

acc2018_seoul = acc2018_df[(acc2018_df['철도구분']=='일반철도')|(acc2018_df['철도구분']=='도시철도')]

# 합계 지연열차수, 합계 최대지연시간을 자치구 별로 합산

acc2018_time = acc2018_seoul.loc[:,['합계지연열차수','합계최대지연시간(분)','군구']]
acc2018_time.groupby(['군구'],as_index=False).sum()

# 상관계수 비교를 위해 해당 연도(2018) 만족도 데이터를 가져옴

AQ36A2 = pd.read_csv('./2018 자치구별 평균 만족도.csv', sep = ',', header = 0, engine = 'python', encoding='cp949')
AQ36A2.columns=['군구','만족도']
AQ36A2.head()

# 두 데이터 프레임을 '군구' column 기준으로 병합

# 결측치는 0 처리

acc2018_merge = pd.merge(AQ36A2,acc2018_time.groupby(['군구'],as_index=False).sum(),how='left',on='군구')
acc2018_merge = acc2018_merge.fillna(0)
acc2018_merge

acc2018_merge.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.267590,0.112354
합계지연열차수,-0.267590,1.000000,0.685064
합계최대지연시간(분),0.112354,0.685064,1.000000


2018년도 만족도(AQ36A2) 와의 상관관계

합계지연열차수  	-0.267590		

합계최대지연시간(분) 0.112354	

# 2. 연도별 흐름으로 자치구별 상관계수를 측정

지하철 사고 통계를 자치구별, 연도별로 볼 수 있도록 변환

In [14]:
acc2020_merge[2020]=2020
acc2020_merge = acc2020_merge.set_index('군구')
acc2020_merge = acc2020_merge.T

In [15]:
acc2020_merge.head()

군구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
만족도,3.976471,3.944121,3.901693,3.988815,3.921301,3.965879,3.961316,3.846832,3.923875,3.881669,...,3.929749,3.908073,3.975179,3.854857,3.865929,3.889494,3.903244,3.868549,3.977827,3.924129
합계지연열차수,29.000000,0.000000,31.000000,0.000000,0.000000,0.000000,15.000000,8.000000,2.000000,1.000000,...,6.000000,0.000000,0.000000,0.000000,267.000000,38.000000,1.000000,0.000000,80.000000,0.000000
합계최대지연시간(분),37.000000,0.000000,15.000000,0.000000,0.000000,0.000000,79.000000,37.000000,43.000000,7.000000,...,41.000000,0.000000,0.000000,0.000000,281.000000,362.000000,65.000000,0.000000,914.000000,0.000000
2020,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,...,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000,2020.000000


In [16]:
# 2019년

acc2019_merge[2019]=2019
acc2019_merge = acc2019_merge.set_index('군구')
acc2019_merge = acc2019_merge.T

display(acc2019_merge.head())

# 2018년

acc2018_merge[2018]=2018
acc2018_merge = acc2018_merge.set_index('군구')
acc2018_merge = acc2018_merge.T

display(acc2018_merge.head())

군구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
만족도,3.922594,3.868676,3.721649,4.126938,3.869782,4.045925,3.840132,4.119898,3.762082,4.031333,...,3.951654,3.522013,4.11571,3.615668,3.666145,3.821656,3.841186,3.793597,3.807898,3.840265
합계지연열차수,1.000000,0.000000,12.000000,78.000000,0.000000,0.000000,66.000000,87.000000,4.000000,2.000000,...,38.000000,5.000000,0.00000,0.000000,2.000000,45.000000,2.000000,0.000000,150.000000,1.000000
합계최대지연시간(분),11.000000,0.000000,92.000000,89.000000,0.000000,27.000000,284.000000,263.000000,26.000000,34.000000,...,90.000000,24.000000,0.00000,0.000000,53.000000,252.000000,59.000000,0.000000,1207.000000,11.000000
2019,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,...,2019.000000,2019.000000,2019.00000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000


군구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
만족도,3.593718,4.184009,4.010471,3.957991,3.935824,4.010063,4.034922,3.765116,3.860453,3.859333,...,3.750909,3.808776,3.869974,3.667553,3.822024,3.917102,3.925463,3.747695,4.086674,3.842105
합계지연열차수,4.000000,0.000000,1.000000,3.000000,0.000000,0.000000,17.000000,94.000000,14.000000,1.000000,...,0.000000,0.000000,46.000000,0.000000,10.000000,82.000000,6.000000,0.000000,1.000000,0.000000
합계최대지연시간(분),16.000000,0.000000,94.000000,287.000000,0.000000,0.000000,166.000000,278.000000,83.000000,11.000000,...,0.000000,0.000000,68.000000,0.000000,86.000000,372.000000,83.000000,0.000000,58.000000,0.000000
2018,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,...,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000,2018.000000


# 강남구

In [17]:
KN_df = pd.concat([acc2020_merge['강남구'],acc2019_merge['강남구'],acc2018_merge['강남구']],axis=1)
KN_df.columns = [2020,2019,2018]
KN_df = KN_df.dropna()
KN_df = KN_df.T
KN_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.976471,29.0,37.0
2019,3.922594,1.0,11.0
2018,3.593718,4.0,16.0


In [18]:
KN_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.528012,0.454463
합계지연열차수,0.528012,1.000000,0.996433
합계최대지연시간(분),0.454463,0.996433,1.000000


# 강동구(X)
3년간 사고 없음

In [19]:
KD_df = pd.concat([acc2020_merge['강동구'],acc2019_merge['강동구'],acc2018_merge['강동구']],axis=1)
KD_df.columns = [2020,2019,2018]
KD_df = KD_df.dropna()
KD_df = KD_df.T
KD_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.944121,0.0,0.0
2019,3.868676,0.0,0.0
2018,4.184009,0.0,0.0


# 강북구

In [20]:
KB_df = pd.concat([acc2020_merge['강북구'],acc2019_merge['강북구'],acc2018_merge['강북구']],axis=1)
KB_df.columns = [2020,2019,2018]
KB_df = KB_df.dropna()
KB_df = KB_df.T
KB_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.901693,31.0,15.0
2019,3.721649,12.0,92.0
2018,4.010471,1.0,94.0


In [21]:
KB_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.227328,-0.11902
합계지연열차수,-0.227328,1.000000,-0.93984
합계최대지연시간(분),-0.119020,-0.939840,1.00000


# 강서구

In [22]:
KS_df = pd.concat([acc2020_merge['강서구'],acc2019_merge['강서구'],acc2018_merge['강서구']],axis=1)
KS_df.columns = [2020,2019,2018]
KS_df = KS_df.dropna()
KS_df = KS_df.T
KS_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.988815,0.0,0.0
2019,4.126938,78.0,89.0
2018,3.957991,3.0,287.0


In [23]:
KS_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.978838,-0.378339
합계지연열차수,0.978838,1.000000,-0.180906
합계최대지연시간(분),-0.378339,-0.180906,1.000000


# 관악구(X)

3년간 사고 없음

In [24]:
GA_df = pd.concat([acc2020_merge['관악구'],acc2019_merge['관악구'],acc2018_merge['관악구']],axis=1)
GA_df.columns = [2020,2019,2018]
GA_df = GA_df.dropna()
GA_df = GA_df.T
GA_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.921301,0.0,0.0
2019,3.869782,0.0,0.0
2018,3.935824,0.0,0.0


# 광진구(X)

In [25]:
GJ_df = pd.concat([acc2020_merge['광진구'],acc2019_merge['광진구'],acc2018_merge['광진구']],axis=1)
GJ_df.columns = [2020,2019,2018]
GJ_df = GJ_df.dropna()
GJ_df = GJ_df.T
GJ_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.965879,0.0,0.0
2019,4.045925,0.0,27.0
2018,4.010063,0.0,0.0


In [26]:
GJ_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,NaN,0.834513
합계지연열차수,NaN,NaN,NaN
합계최대지연시간(분),0.834513,NaN,1.000000


# 구로구

In [27]:
KR_df = pd.concat([acc2020_merge['구로구'],acc2019_merge['구로구'],acc2018_merge['구로구']],axis=1)
KR_df.columns = [2020,2019,2018]
KR_df = KR_df.dropna()
KR_df = KR_df.T
KR_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.961316,15.0,79.0
2019,3.840132,66.0,284.0
2018,4.034922,17.0,166.0


In [28]:
KR_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.00000,-0.913850,-0.682210
합계지연열차수,-0.91385,1.000000,0.920325
합계최대지연시간(분),-0.68221,0.920325,1.000000


# 금천구

In [29]:
KC_df = pd.concat([acc2020_merge['금천구'],acc2019_merge['금천구'],acc2018_merge['금천구']],axis=1)
KC_df.columns = [2020,2019,2018]
KC_df = KC_df.dropna()
KC_df = KC_df.T
KC_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.846832,8.0,37.0
2019,4.119898,87.0,263.0
2018,3.765116,94.0,278.0


In [30]:
KC_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.226541,0.243815
합계지연열차수,0.226541,1.000000,0.999842
합계최대지연시간(분),0.243815,0.999842,1.000000


# 노원구

In [31]:
NW_df = pd.concat([acc2020_merge['노원구'],acc2019_merge['노원구'],acc2018_merge['노원구']],axis=1)
NW_df.columns = [2020,2019,2018]
NW_df = NW_df.dropna()
NW_df = NW_df.T
NW_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.923875,2.0,43.0
2019,3.762082,4.0,26.0
2018,3.860453,14.0,83.0


In [32]:
NW_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.032097,0.406655
합계지연열차수,-0.032097,1.000000,0.900059
합계최대지연시간(분),0.406655,0.900059,1.000000


# 도봉구

In [33]:
DB_df = pd.concat([acc2020_merge['도봉구'],acc2019_merge['도봉구'],acc2018_merge['도봉구']],axis=1)
DB_df.columns = [2020,2019,2018]
DB_df = DB_df.dropna()
DB_df = DB_df.T
DB_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.881669,1.0,7.0
2019,4.031333,2.0,34.0
2018,3.859333,1.0,11.0


In [34]:
DB_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.992845,0.967060
합계지연열차수,0.992845,1.000000,0.990536
합계최대지연시간(분),0.967060,0.990536,1.000000


# 동대문구

In [35]:
DDM_df = pd.concat([acc2020_merge['동대문구'],acc2019_merge['동대문구'],acc2018_merge['동대문구']],axis=1)
DDM_df.columns = [2020,2019,2018]
DDM_df = DDM_df.dropna()
DDM_df = DDM_df.T
DDM_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.872775,12.0,50.0
2019,3.775522,18.0,93.0
2018,3.608387,96.0,139.0


In [36]:
DDM_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.952911,-0.991304
합계지연열차수,-0.952911,1.000000,0.904719
합계최대지연시간(분),-0.991304,0.904719,1.000000


# 동작구

In [37]:
DJ_df = pd.concat([acc2020_merge['동작구'],acc2019_merge['동작구'],acc2018_merge['동작구']],axis=1)
DJ_df.columns = [2020,2019,2018]
DJ_df = DJ_df.dropna()
DJ_df = DJ_df.T
DJ_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.905250,0.0,0.0
2019,3.847588,28.0,70.0
2018,3.796041,8.0,19.0


In [38]:
DJ_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.308249,-0.293483
합계지연열차수,-0.308249,1.000000,0.999880
합계최대지연시간(분),-0.293483,0.999880,1.000000


# 마포구

In [39]:
MP_df = pd.concat([acc2020_merge['마포구'],acc2019_merge['마포구'],acc2018_merge['마포구']],axis=1)
MP_df.columns = [2020,2019,2018]
MP_df = MP_df.dropna()
MP_df = MP_df.T
MP_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.955600,0.0,0.0
2019,4.114171,8.0,106.0
2018,3.789303,19.0,26.0


In [40]:
MP_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.587719,0.714511
합계지연열차수,-0.587719,1.000000,0.146109
합계최대지연시간(분),0.714511,0.146109,1.000000


# 서대문구

In [41]:
SDM_df = pd.concat([acc2020_merge['서대문구'],acc2019_merge['서대문구'],acc2018_merge['서대문구']],axis=1)
SDM_df.columns = [2020,2019,2018]
SDM_df = SDM_df.dropna()
SDM_df = SDM_df.T
SDM_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.947479,0.0,0.0
2019,3.697817,2.0,0.0
2018,3.798658,3.0,21.0


In [42]:
SDM_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.734025,-0.110280
합계지연열차수,-0.734025,1.000000,0.755929
합계최대지연시간(분),-0.110280,0.755929,1.000000


# 서초구

In [43]:
SC_df = pd.concat([acc2020_merge['서초구'],acc2019_merge['서초구'],acc2018_merge['서초구']],axis=1)
SC_df.columns = [2020,2019,2018]
SC_df = SC_df.dropna()
SC_df = SC_df.T
SC_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.918002,0.0,0.0
2019,3.905325,0.0,0.0
2018,3.830069,3.0,64.0


In [44]:
SC_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.991069,-0.991069
합계지연열차수,-0.991069,1.000000,1.000000
합계최대지연시간(분),-0.991069,1.000000,1.000000


# 성동구

In [45]:
SD_df = pd.concat([acc2020_merge['성동구'],acc2019_merge['성동구'],acc2018_merge['성동구']],axis=1)
SD_df.columns = [2020,2019,2018]
SD_df = SD_df.dropna()
SD_df = SD_df.T
SD_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.929749,6.0,41.0
2019,3.951654,38.0,90.0
2018,3.750909,0.0,0.0


In [46]:
SD_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.696569,0.889177
합계지연열차수,0.696569,1.000000,0.947670
합계최대지연시간(분),0.889177,0.947670,1.000000


# 성북구

In [47]:
SB_df = pd.concat([acc2020_merge['성북구'],acc2019_merge['성북구'],acc2018_merge['성북구']],axis=1)
SB_df.columns = [2020,2019,2018]
SB_df = SB_df.dropna()
SB_df = SB_df.T
SB_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.908073,0.0,0.0
2019,3.522013,5.0,24.0
2018,3.808776,0.0,0.0


In [48]:
SB_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.968848,-0.968848
합계지연열차수,-0.968848,1.000000,1.000000
합계최대지연시간(분),-0.968848,1.000000,1.000000


# 송파구

In [49]:
SP_df = pd.concat([acc2020_merge['송파구'],acc2019_merge['송파구'],acc2018_merge['송파구']],axis=1)
SP_df.columns = [2020,2019,2018]
SP_df = SP_df.dropna()
SP_df = SP_df.T
SP_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.975179,0.0,0.0
2019,4.115710,0.0,0.0
2018,3.869974,46.0,68.0


In [50]:
SP_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.821701,-0.821701
합계지연열차수,-0.821701,1.000000,1.000000
합계최대지연시간(분),-0.821701,1.000000,1.000000


# 양천구(X)

In [51]:
YC_df = pd.concat([acc2020_merge['양천구'],acc2019_merge['양천구'],acc2018_merge['양천구']],axis=1)
YC_df.columns = [2020,2019,2018]
YC_df = YC_df.dropna()
YC_df = YC_df.T
YC_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.854857,0.0,0.0
2019,3.615668,0.0,0.0
2018,3.667553,0.0,0.0


# 영등포구

In [52]:
YDP_df = pd.concat([acc2020_merge['영등포구'],acc2019_merge['영등포구'],acc2018_merge['영등포구']],axis=1)
YDP_df.columns = [2020,2019,2018]
YDP_df = YDP_df.dropna()
YDP_df = YDP_df.T
YDP_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.865929,267.0,281.0
2019,3.666145,2.0,53.0
2018,3.822024,10.0,86.0


In [53]:
YDP_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.689486,0.763394
합계지연열차수,0.689486,1.000000,0.994198
합계최대지연시간(분),0.763394,0.994198,1.000000


# 용산구

In [54]:
YS_df = pd.concat([acc2020_merge['용산구'],acc2019_merge['용산구'],acc2018_merge['용산구']],axis=1)
YS_df.columns = [2020,2019,2018]
YS_df = YS_df.dropna()
YS_df = YS_df.T
YS_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.889494,38.0,362.0
2019,3.821656,45.0,252.0
2018,3.917102,82.0,372.0


In [55]:
YS_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.613044,0.978088
합계지연열차수,0.613044,1.000000,0.435130
합계최대지연시간(분),0.978088,0.435130,1.000000


# 은평구

In [56]:
EP_df = pd.concat([acc2020_merge['은평구'],acc2019_merge['은평구'],acc2018_merge['은평구']],axis=1)
EP_df.columns = [2020,2019,2018]
EP_df = EP_df.dropna()
EP_df = EP_df.T
EP_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.903244,1.0,65.0
2019,3.841186,2.0,59.0
2018,3.925463,6.0,83.0


In [57]:
EP_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,0.556887,0.853842
합계지연열차수,0.556887,1.000000,0.907841
합계최대지연시간(분),0.853842,0.907841,1.000000


# 종로구(X)

In [58]:
JR_df = pd.concat([acc2020_merge['종로구'],acc2019_merge['종로구'],acc2018_merge['종로구']],axis=1)
JR_df.columns = [2020,2019,2018]
JR_df = JR_df.dropna()
JR_df = JR_df.T
JR_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.868549,0.0,0.0
2019,3.793597,0.0,0.0
2018,3.747695,0.0,0.0


# 중구

In [59]:
JG_df = pd.concat([acc2020_merge['중구'],acc2019_merge['중구'],acc2018_merge['중구']],axis=1)
JG_df.columns = [2020,2019,2018]
JG_df = JG_df.dropna()
JG_df = JG_df.T
JG_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.977827,80.0,914.0
2019,3.807898,150.0,1207.0
2018,4.086674,1.0,58.0


In [60]:
JG_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.987116,-0.920465
합계지연열차수,-0.987116,1.000000,0.971140
합계최대지연시간(분),-0.920465,0.971140,1.000000


# 중랑구

In [61]:
JL_df = pd.concat([acc2020_merge['중랑구'],acc2019_merge['중랑구'],acc2018_merge['중랑구']],axis=1)
JL_df.columns = [2020,2019,2018]
JL_df = JL_df.dropna()
JL_df = JL_df.T
JL_df.head()

,만족도,합계지연열차수,합계최대지연시간(분)
2020,3.924129,0.0,0.0
2019,3.840265,1.0,11.0
2018,3.842105,0.0,0.0


In [62]:
JL_df.corr(method='pearson')

,만족도,합계지연열차수,합계최대지연시간(분)
만족도,1.000000,-0.516543,-0.516543
합계지연열차수,-0.516543,1.000000,1.000000
합계최대지연시간(분),-0.516543,1.000000,1.000000


# 총평

1. 지연 열차 수, 지연시간 등의 차이는 자치구 별 만족도에 영향을 주지 못했다.


2. 지하철 사고, 점검으로 인한 지연이 거의 없는 지역에서는 만족도에 영향을 끼치는 경우가 많지만
    평소 시간 지연이 자주 일어나는 지역은 만족도에 영향을 끼치지 못하는 경우가 많다.